In [ ]:
!pip install neuralforecast utilsforecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

from neuralforecast.core import NeuralForecast
from neuralforecast.models import MLP, MLPMultivariate
from neuralforecast.losses.pytorch import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def plot_metrics_bar(eval_df):
    methods = eval_df.columns[1:]
    values = eval_df.iloc[0].values[1:]

    sorted_data = sorted(zip(methods, values), key=lambda x: x[1], reverse=True)
    methods_sorted, values_sorted = zip(*sorted_data)

    plt.figure(figsize=(10, 6))
    bars = plt.bar(methods_sorted, values_sorted)

    for bar, value in zip(bars, values_sorted):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                 f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

    plt.xlabel('Methods')
    plt.ylabel('Mean absolute error (MAE)')
    plt.tight_layout()

    plt.show()

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/youtube_tutorials/refs/heads/main/data/daily_sales_french_bakery.csv"
df = pd.read_csv(DATA_URL, parse_dates=["ds"])
df = df.groupby('unique_id').filter(lambda x: len(x) >= 28)
df.head()

In [ ]:
plot_series(df=df, ids=["BAGUETTE", "CROISSANT"], palette="viridis")

In [ ]:
plot_series(df=df, ids=["BAGUETTE", "CROISSANT"], max_insample_length=56, palette="viridis")

In [ ]:
unique_ids = ["BAGUETTE", "CROISSANT"]
df = df[df["unique_id"].isin(unique_ids)]

## Forecasting and cross-validation

In [ ]:
HORIZON = 7

In [ ]:
models = [
    MLP(
        h=HORIZON,
        input_size=14,
        futr_exog_list=["unit_price"],
        num_layers=2,
        hidden_size=1024,
        loss=MAE(),
        valid_loss=MSE(),
        max_steps=1000,
        early_stop_patience_steps=3,
        val_check_steps=100,
        scaler_type="identity"
        ),
    MLPMultivariate(
        h=HORIZON,
        input_size=14,
        n_series=2,
        futr_exog_list=["unit_price"],
        num_layers=2,
        hidden_size=1024,
        loss=MAE(),
        valid_loss=MSE(),
        max_steps=1000,
        early_stop_patience_steps=3,
        val_check_steps=100,
        scaler_type="identity"
    )
]

In [ ]:
nf = NeuralForecast(models=models, freq="D")
nf.fit(df=df, val_size=HORIZON)

In [ ]:
last_date = df['ds'].max()
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=HORIZON, freq='D')

# Get the last unit_price for each unique_id
last_unit_prices = df.groupby('unique_id')['unit_price'].last()

futr_df = []
for uid in unique_ids:
    last_price = last_unit_prices[uid]
    for date in future_dates:
        futr_df.append({
            'unique_id': uid,
            'ds': date,
            'unit_price': last_price
        })

futr_df = pd.DataFrame(futr_df)
futr_df

In [ ]:
preds = nf.predict(futr_df=futr_df)

In [ ]:
plot_series(df=df, forecasts_df=preds, max_insample_length=56, palette="viridis")

### Cross-validation

In [ ]:
cv_df = nf.cross_validation(
    df=df,
    step_size=HORIZON,
    val_size=HORIZON,
    test_size=8*HORIZON,
    refit=False,
    n_windows=None,
)

evaluation = evaluate(
    cv_df.drop(["cutoff"], axis=1),
    metrics=[mae, rmse],
)

evaluation = evaluation.drop(['unique_id'], axis=1).groupby('metric').mean().reset_index()
evaluation

In [ ]:
plot_metrics_bar(evaluation)

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df.drop(["y", "cutoff"], axis=1),
    max_insample_length=140,
    palette="viridis"
)

## Probabilistic forecasting
### Loss function

In [ ]:
models = [
    MLP(
        h=HORIZON,
        input_size=14,
        futr_exog_list=["unit_price"],
        num_layers=2,
        hidden_size=1024,
        loss=HuberMQLoss(level=[80]),
        valid_loss=HuberMQLoss(level=[80]),
        max_steps=1000,
        early_stop_patience_steps=3,
        val_check_steps=100,
        scaler_type="identity"
        )
]

nf = NeuralForecast(models=models, freq="D")

cv_df_mqloss = nf.cross_validation(
    df=df,
    step_size=HORIZON,
    val_size=HORIZON,
    test_size=8*HORIZON,
    refit=False,
    n_windows=None,
    level=[80]
)

cv_df_mqloss = cv_df_mqloss.rename(columns={"MLP-median": "MLP"})

cv_df_mqloss.head()

In [ ]:
metrics = [
    scaled_crps
]

evaluation = evaluate(
    cv_df_mqloss.drop(["ds", "cutoff"], axis=1),
    metrics=metrics,
    models=["MLP"],
    level=[80]
)
evaluation = evaluation.drop(['unique_id'], axis=1).groupby('metric').mean().reset_index()
evaluation

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df_mqloss.drop(["y", "cutoff"], axis=1),
    # models=["MLP"],
    max_insample_length=140,
    level=[80],
    palette="viridis"
)

### Conformal predictions

In [ ]:
from neuralforecast.utils import PredictionIntervals

In [ ]:
prediction_intervals = PredictionIntervals()

models = [
    MLP(
        h=HORIZON,
        input_size=14,
        futr_exog_list=["unit_price"],
        num_layers=2,
        hidden_size=1024,
        loss=MAE(),
        max_steps=400,
        scaler_type="identity"
        )
]

nf = NeuralForecast(models=models, freq="D")

cv_df_conformal = nf.cross_validation(
    df=df,
    step_size=HORIZON,
    test_size=8*HORIZON,
    refit=True,
    n_windows=None,
    prediction_intervals=prediction_intervals,
    level=[80]
)

cv_df_conformal.head()

In [ ]:
metrics = [
    scaled_crps
]

evaluation = evaluate(
    cv_df_conformal.drop(["ds", "cutoff"], axis=1),
    metrics=metrics,
    models=["MLP"],
    level=[80]
)
evaluation = evaluation.drop(['unique_id'], axis=1).groupby('metric').mean().reset_index()
evaluation

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df_conformal.drop(["y", "cutoff"], axis=1),
    # models=["MLP"],
    max_insample_length=140,
    level=[80],
    palette="viridis"
)

## Training on large datasets

In [ ]:
import os

In [ ]:
output_dir = "partitioned_data"
os.makedirs(output_dir, exist_ok=True)

# Save partitioned by unique_id
df.to_parquet(output_dir, partition_cols=['unique_id'], index=False)

# Get list of partition directories
files_list = [f"{output_dir}/{dir_name}" for dir_name in os.listdir(output_dir)]

In [ ]:
files_list

In [ ]:
models = [
    MLP(
        h=HORIZON,
        input_size=14,
        futr_exog_list=["unit_price"],
        num_layers=2,
        hidden_size=1024,
        loss=MAE(),
        max_steps=400,
        scaler_type="identity"
        )
]

nf = NeuralForecast(models=models, freq="D")
nf.fit(df=files_list)

In [ ]:
input_df = df.groupby('unique_id').tail(14)

preds = nf.predict(df=input_df, futr_df=futr_df)

In [ ]:
plot_series(df=df, forecasts_df=preds, max_insample_length=56, palette="viridis")

## Hyperparameter optimization

In [ ]:
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch

from neuralforecast.auto import AutoMLP

In [ ]:
mlp_config = {
    "max_steps": 100,
    "input_size": tune.choice([7, 14]),
    "hidden_size": tune.choice([512, 1024]),
    "num_layers": tune.choice([2, 3]),
    "learning_rate": tune.loguniform(1e-5, 1e-1),
}

In [ ]:
model = AutoMLP(
    h=HORIZON,
    loss=MAE(),
    config=mlp_config,
    search_alg=HyperOptSearch(),
    backend='ray',
    num_samples=10
)

In [ ]:
nf = NeuralForecast(models=[model], freq='D')
nf.fit(df=df.drop(columns=["unit_price"]), val_size=28)

In [ ]:
results = nf.models[0].results.get_dataframe()

config_cols = [col for col in results.columns if col.startswith('config')]
columns_to_keep = ['loss', 'train_loss'] + config_cols
existing_columns = [col for col in columns_to_keep if col in results.columns]
filtered_results = results[existing_columns]
best_runs = filtered_results.sort_values('loss', ascending=True).head(5)
best_runs